In [22]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from torch.utils.data import Dataset
import torch.nn as nn

In [23]:
# Load dataset with detected encoding
df = pd.read_csv('all-data-engineering-quiz-utf.csv')

In [24]:
# Combine relevant text fields for input, joining reasoning fields if they are arrays
df['input_text'] = (
    df['Question'] + " " +
    df['Student Answer'] + " " +
    df['Ideal Answer'] + 
    " Grammar Reasoning: " + df['Grammar Reasoning'].apply(lambda x: ' | '.join(eval(x)) if isinstance(x, str) and x.startswith('[') else str(x)) + 
    " Structure Reasoning: " + df['Structure Reasoning'].apply(lambda x: ' | '.join(eval(x)) if isinstance(x, str) and x.startswith('[') else str(x))
)

# Define target columns
df['content_relevancy_score'] = df['Content Relevancy Score'].astype(float)
df['grammar_score'] = df['Grammar Score'].astype(float)
df['structure_score'] = df['Structure Score'].astype(float)

# Scale scores to range from 0 to 1 (optional for normalized regression)
df['content_relevancy_score'] /= 3  # Normalize content relevancy score to 0-1
df['grammar_score'] /= 5            # Normalize grammar score to 0-1
df['structure_score'] /= 5          # Normalize structure score to 0-1

# Train-test split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)



In [25]:
# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [26]:
# Custom Dataset Class
class EssayDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        text = self.dataframe.iloc[idx]['input_text']
        content_score = self.dataframe.iloc[idx]['content_relevancy_score']
        grammar_score = self.dataframe.iloc[idx]['grammar_score']
        structure_score = self.dataframe.iloc[idx]['structure_score']
        
        # Tokenize input text
        inputs = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        # Combine scores into a single tensor for regression
        labels = torch.tensor([content_score, grammar_score, structure_score], dtype=torch.float)
        
        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'labels': labels
        }

# Create datasets
max_length = 512
train_dataset = EssayDataset(train_df, tokenizer, max_length)
val_dataset = EssayDataset(val_df, tokenizer, max_length)


In [27]:
# Custom BERT Model for Multi-output Regression
class BertForMultiOutputRegression(nn.Module):
    def __init__(self):
        super(BertForMultiOutputRegression, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.regressor = nn.Linear(self.bert.config.hidden_size, 3)  # 3 outputs: content, grammar, structure

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        logits = self.regressor(pooled_output)

        loss = None
        if labels is not None:
            loss_fct = nn.MSELoss()
            loss = loss_fct(logits, labels)

        return {'loss': loss, 'logits': logits} if loss is not None else {'logits': logits}

# Initialize the model
model = BertForMultiOutputRegression()

In [28]:
# Define custom metrics function for evaluation
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.squeeze()
    mse = mean_squared_error(labels, predictions, squared=False)
    return {'rmse': mse}

from transformers import AdamW, Trainer, TrainingArguments

# Adjusted Training Arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-6,              # Lower learning rate
    per_device_train_batch_size=8,   # Reduce batch size if needed for memory
    per_device_eval_batch_size=8,
    num_train_epochs=50,             # Increase epochs
    weight_decay=0.02,               # Increased weight decay
    load_best_model_at_end=True,
    metric_for_best_model="rmse",
    greater_is_better=False,
    logging_dir='./logs',
    logging_steps=10,
    gradient_accumulation_steps=2    # Simulate larger batch size
)


# Initialize Trainer with custom optimizer if necessary
optimizer = AdamW(model.parameters(), lr=5e-6, weight_decay=0.02)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, None)     # Custom optimizer if needed
)

# Fine-tune the model
trainer.train()

d:\research_models\handwritten_ocr\handwritten_ocr\.venv\Lib\site-packages\transformers\training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
d:\research_models\handwritten_ocr\handwritten_ocr\.venv\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  2%|▏         | 2/100 [01:16<1:01:34, 37.70s/it]

d:\research_models\handwritten_ocr\handwritten_ocr\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
                                                

{'eval_loss': 0.46533769369125366, 'eval_rmse': 0.5950549244880676, 'eval_runtime': 6.0791, 'eval_samples_per_second': 1.645, 'eval_steps_per_second': 0.329, 'epoch': 0.8}


  5%|▌         | 5/100 [02:55<52:27, 33.13s/it]  

d:\research_models\handwritten_ocr\handwritten_ocr\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
                                               
                                    

  5%|▌         | 5/100 [03:01<52:27, 33.13s/it]



{'eval_loss': 0.3711877465248108, 'eval_rmse': 0.5409286618232727, 'eval_runtime': 5.8835, 'eval_samples_per_second': 1.7, 'eval_steps_per_second': 0.34, 'epoch': 2.0}


  7%|▋         | 7/100 [04:21<59:38, 38.47s/it]

d:\research_models\handwritten_ocr\handwritten_ocr\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
                                               
                                    

  7%|▋         | 7/100 [04:35<59:38, 38.47s/it]



...



d:\research_models\handwritten_ocr\handwritten_ocr\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
                                                 
                                      

100%|██████████| 100/100 [1:00:02<00:00, 31.89s/it]



{'eval_loss': 0.022415360435843468, 'eval_rmse': 0.14900733530521393, 'eval_runtime': 6.1482, 'eval_samples_per_second': 1.626, 'eval_steps_per_second': 0.325, 'epoch': 40.0}


                                                   
100%|██████████| 100/100 [1:00:04<00:00, 36.05s/it]

{'train_runtime': 3604.737, 'train_samples_per_second': 0.499, 'train_steps_per_second': 0.028, 'train_loss': 0.13698517948389052, 'epoch': 40.0}


TrainOutput(global_step=100, training_loss=0.13698517948389052, metrics={'train_runtime': 3604.737, 'train_samples_per_second': 0.499, 'train_steps_per_second': 0.028, 'total_flos': 0.0, 'train_loss': 0.13698517948389052, 'epoch': 40.0})

In [31]:
# Evaluate the model
metrics = trainer.evaluate()
print(metrics)

100%|██████████| 2/2 [00:01<00:00,  1.70it/s]d:\research_models\handwritten_ocr\handwritten_ocr\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
100%|██████████| 2/2 [00:01<00:00,  1.70it/s]

{'eval_loss': 0.021104011684656143, 'eval_rmse': 0.1450696587562561, 'eval_runtime': 6.168, 'eval_samples_per_second': 1.621, 'eval_steps_per_second': 0.324, 'epoch': 40.0}


In [30]:
# Save the fine-tuned model
trainer.save_model('./essay_scoring_model')
tokenizer.save_pretrained('./essay_scoring_model')

('./essay_scoring_model\\tokenizer_config.json',
 './essay_scoring_model\\special_tokens_map.json',
 './essay_scoring_model\\vocab.txt',
 './essay_scoring_model\\added_tokens.json')